In [2]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response


# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:テスト'

テストが正常に行われました。どのようなお手伝いができますか？

---ご利用ありがとうございました！---


In [10]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、2025年の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
2025年の東京駅に関連するイベント情報については、いくつかの興味深いイベントが計画されています。以下は主要なイベントです。

1. **夏の風物詩「昇路灯2025」**
   - 開催期間：2025年8月29日（金）〜9月7日（日） 
   - 場所：東京駅八重洲口のグランルーフガーデン
   - 内容：100基の灯籠が設置され、幻想的な光の空間が創出されます。飲食ブースも設置され、特別な体験を提供します。
   - [詳細情報](https://www.tokyofes.info/2025/yoimichi-torou-2025/)

2. **「昇路灯2025」関連のイベント**
   - 今年のテーマは「八重洲の花火」で、大規模な花火が打ち上げられます。幻想的な光と音の演出が予定されています。

3. **MARUNOUCHI STREET PARK 2025 Winter**
   - 開催期間：2025年11月13日（木）〜12月25日（木）
   - 場所：丸の内エリア
   - 内容：光の祭典や市場などのイベントが行われ、多くの飲食店やアートが楽しめる空間になります。
   - [詳細情報](https://marunouchi-streetpark.com/)

これらのイベントは、東京駅周辺や丸の内エリアで行われ、訪れる人々に特別な体験を提供することが期待されています。


In [5]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

    
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)


------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- 東京都の面積は約2,194平方キロメートルです。
- 沖縄県の面積は約2,271平方キロメートルです。

したがって、沖縄県が東京都よりも広いことがわかります。


In [4]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})


# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'content': '開催中 2025年12月20日(土)～2026年2月22日(日). 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都). CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ). 日比谷駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 京橋駅(東京都). 開催中 2026年1月2日(金)～3月22日(日). 二重橋前駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 日比谷駅(東京都), 銀座一丁目駅(東京都). * 歴史リアル謎解きゲーム「謎の城」in 日本橋「発明家／人斬り-平賀源内-」. 開催中 2025年9月1日(月)～2026年3月1日(日). 日本橋駅(東京都), 京橋駅(東京都), 東京駅(東京都), 宝町駅(東京都), 三越前駅(東京都). * MIDTOWN YAESU CHRISTMAS 2025 (ミッドタウン八重洲クリスマス2025). 終了間近 2025年11月13日(木)～2026年2月15日(日). 京橋駅(東京都), 東京駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都). * 丸の内から御食国「敦賀・若狭」へ 美し物-うましもの-グルメフェア. 開催中 2026年2月2日(月)～21日(土). 二重橋前駅(東京都), 有楽町駅(東京都), 東京駅(東京都), 日比谷駅(東京都). 終了間近 2025年11月13日(木)～2026年2月15日(日). 二重橋前〈丸の内〉駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 日比谷駅(東京都), 銀座一丁目駅(東京都). * Masking Tape Jamboree in KITTE2026. 終了間近 2026年2月11日(水)～13日(金). 東京駅(東京都), 二重橋前駅(東京都), 有楽町駅(東京都), 京橋駅(東京都), 銀座一丁目駅(東京都). 東京駅(東京都), 二重橋前駅